In [11]:
import codecs
import re
import random
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score

In [44]:
#将人工标注的数据转化成crf模型格式
def get_crf_data(line, words):
    indexs = []
    #for m in re.finditer(r'\[(.*?)\]',string):
    #    indexs += [m.start(0),m.end(0)]
    pre,flag,res = '',False,[]
    for i in range(len(line)):
        word = line[i]
        if word != '[' or word != ']':
            words.add(word)
        if word == '[':
            pre = word
            continue
        elif pre == '[':
            res.append([word,'B'])
            flag = True
        elif word == ']':
            res[-1][-1] = 'E'
            flag = False
        elif flag:
            res.append([word, 'M'])
        else:
            res.append([word, 'O'])
        pre = word
    return res

def process_file(path):
    res, max_len, words = [], 0, set()
    with codecs.open(path) as fin:
        for line in fin:
            line = line.strip()
            count = 0
            for word, label in get_crf_data(line, words):
                res.append(word + ' ' + label + '\n')
                count += 1
            if count > max_len: max_len = count
            res.append('\n')
    return max_len, words, res

def write_data(lines,out_path):
    with open(out_path,'w') as fout:
        for line in lines:
            fout.write(line)
def write_words(words, out_path):
    with codecs.open(out_path, 'w', encoding='gbk') as fout:
        for word in words:
            fout.write(word + '\n')

In [81]:
max_len, words, res = process_file('data/bendibao.txt')
write_data(res, 'data/bendibao.crf')
print(max_len)
write_words(words, 'data/words.txt')
#print(word2id)
#print(id2word)
#get_crf_data('杭州[去哪吃夜宵]？[杭州吃夜宵好去处]')

37


In [18]:
string = 'abc[bde]ioe[oeds]in'

In [19]:
re.findall(r'\[(.*?)\]',string)

['bde', 'oeds']

In [20]:
res = re.finditer(r'\[(.*?)\]',string)

In [21]:
for m in res:
    print(m.group(0),m.start(0),m.end(0))

[bde] 3 8
[oeds] 11 17


In [12]:
#tools，工具类
#将CRf数据格式进行处理，完成数据的填充，以及batch_size的数据集获取
class Tools:
    def __init__(self, seq_length):
        self.pad_word = '<PAD>'
        self.pad_tag = 'O'
        self.seq_length = seq_length
        self.words = []
        self.word2id = {}
        self.id2word = {}
        self.tag2id = {}
        
        self.get_tag2id()
        self.load_words('data/words.txt')
        self.get_word_dict()
    
    def get_tag2id(self):
        self.tag2id = {'O':[1.,.0,.0,.0], 'B':[.0,1.,.0,.0], 'M':[.0,.0,1.,.0], 'E':[.0,.0,.0,1.]}
        
    def load_words(self, path):
        
        with codecs.open(path, 'r', encoding='gbk') as fin:
            for line in fin:
                word = line.strip()
                if len(word) == 0: continue
                self.words.append(word)
                
    def get_word_dict(self):
        for index, word in enumerate(self.words, 1):
            self.word2id[word] = index
            self.id2word[index] = word
        self.word2id[self.pad_word] = 0
        self.id2word[0] = self.pad_word
    
    def padding_sequence(self, sequence, seq_length):
        train_list, tag_list = [], []
        #print(sequence)
        #print(seq_length)
        for i in range(seq_length):
            if i >= len(sequence):
                train_list.append(self.word2id[self.pad_word])
                tag_list.append(self.tag2id[self.pad_tag])
            else:
                try:
                    train_list.append(self.word2id[sequence[i][0]])
                    tag_list.append(self.tag2id[sequence[i][1]])
                except Exception as e:
                    print(str(e))
                    print(sequence)
        return train_list, tag_list
    
    def process_file(self, path):
        X, Y, sequence = [], [], []
        with codecs.open(path, 'r', encoding='gbk') as fin:
            for line in fin:
                if line == '\n':
                    sub_x, sub_y = self.padding_sequence(sequence, self.seq_length)
                    X.append(sub_x)
                    Y.append(sub_y)
                    sequence = []
                else:
                    items = line.strip().split(' ')
                    if len(items) > 1:
                        sequence.append(items)
            return np.array(X), np.array(Y)
    
    def next_batch(self,X, Y, batch_size=64):
        length = len(X)
        num_batch = length // batch_size
        indexs = list(range(length))
        random.shuffle(indexs)
        x_shuffle = X[indexs]
        y_shuffle = Y[indexs]
        
        for i in range(num_batch):
            start = i * batch_size
            end = min((i+1) * batch_size, length)
            yield x_shuffle[start:end], y_shuffle[start:end]

In [13]:
tools = Tools(37)
X,Y = tools.process_file('data/bendibao.train')
X_val, Y_val = tools.process_file('data/bendibao.test')
print(len(tools.word2id))

1745


In [15]:
print(len(X), len(X_val))
print(tools.id2word)

4667 1444
{1: '太', 2: '哈', 3: '吹', 4: '声', 5: '海', 6: '肇', 7: '浣', 8: '件', 9: '可', 10: '价', 11: '木', 12: '则', 13: '瀛', 14: '气', 15: '涪', 16: '言', 17: '轿', 18: '沃', 19: '名', 20: '径', 21: '雍', 22: '烘', 23: '期', 24: '树', 25: '庆', 26: '十', 27: '芳', 28: '夜', 29: '坑', 30: '材', 31: '无', 32: '凰', 33: '溪', 34: '堰', 35: '纪', 36: '青', 37: '僻', 38: '秦', 39: '住', 40: '懂', 41: '即', 42: '吃', 43: '想', 44: '灰', 45: '止', 46: '昆', 47: '远', 48: '成', 49: '植', 50: '繁', 51: '君', 52: '致', 53: '1', 54: '畅', 55: '逸', 56: '早', 57: '圳', 58: '完', 59: '强', 60: '鹭', 61: '狼', 62: '挨', 63: '洋', 64: '桠', 65: '天', 66: '那', 67: '妹', 68: '复', 69: '激', 70: '灵', 71: '槎', 72: '嵘', 73: '寒', 74: '赤', 75: '野', 76: '沽', 77: '释', 78: '滨', 79: '因', 80: '化', 81: '枇', 82: '宗', 83: '柚', 84: '轨', 85: '负', 86: '黎', 87: '冰', 88: '交', 89: '堪', 90: '汉', 91: '增', 92: '速', 93: '兜', 94: '蜡', 95: '朗', 96: '稻', 97: '望', 98: '结', 99: '逗', 100: '雷', 101: '哪', 102: '素', 103: '曹', 104: '降', 105: '氧', 106: '珍', 107: '鸭', 108: '塑', 109: '个', 110: '荡

In [5]:
'''
count = 0
for x, y in tools.next_batch(X, Y, 64):
    count += 1
    if count == 2: break
    print(x)
    print(y)
'''

'\ncount = 0\nfor x, y in tools.next_batch(X, Y, 64):\n    count += 1\n    if count == 2: break\n    print(x)\n    print(y)\n'

In [16]:
#构建lstm 网络
class lstm_model(object):
    def __init__(self):
        self.batch_size = 64
        self.seq_length = 37
        self.num_classes = 4
        self.lr = 0.02
        self.lr_decay = 0.9
        self.dropout_keep_pro = 0.8
        self.num_epoch = 20
        self.hidden_dim = 128
        self.num_layer = 2
        self.embedding_dim = 64
        self.vocab_size = 1745
        
        self.run()
        
    def run(self):
        #inpux
        tf.reset_default_graph()
        self.input_x = tf.placeholder(dtype=tf.int32, shape=[None, self.seq_length], name='input_x')
        self.input_y = tf.placeholder(dtype=tf.float32, shape=[None, self.seq_length, self.num_classes], name='input_y')
        self.dropout_keep_pro = tf.placeholder(dtype=tf.float32, name='keep_pro')
        
        #input_y 变形
        self.labels = tf.reshape(self.input_y, [-1, self.num_classes])
        
        #lstm
        def lstm_cell():
            return tf.contrib.rnn.LSTMCell(self.hidden_dim)
        
        def dropout():
            cell = lstm_cell()
            return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.dropout_keep_pro)
        
        def get_weight(shape):
            return tf.Variable(tf.random_normal(shape=shape, stddev=0.1))
        
        #embedding
        with tf.name_scope('embedding'):
            embedding = tf.get_variable('embedding', shape=[self.vocab_size, self.embedding_dim], dtype=tf.float32)
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)
    
        with tf.name_scope('bi_lstm'):
            #cells = [dropout() for i in range(self.num_layer)]
            #rnn_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
            #fw_cell, bw_cell = lstm_cell(), lstm_cell
            # lstm模型　正方向传播的RNN
            lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_dim, forget_bias=1.0)
            # 反方向传播的RNN
            lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_dim, forget_bias=1.0)
            #print(fw_cell)
            (fw_outputs, bw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, embedding_inputs, dtype=tf.float32)
            outputs = tf.concat([fw_outputs, bw_outputs], axis= 2)
            
            #outputs,_ = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_inputs, dtype=tf.float32)
            print(embedding_inputs.shape)
            print(outputs.shape)
            print(outputs[:,-1,:].shape)
            
            #改变形状
            outputs = tf.reshape(outputs, [-1, 2 * self.hidden_dim])
        
        with tf.name_scope('full_layer'):
            weight1 = get_weight(shape=[2 * self.hidden_dim, self.hidden_dim])
            biases = tf.Variable(tf.constant(0.1,dtype=tf.float32, shape = [self.hidden_dim]))
            fc1 = tf.matmul(outputs, weight1) + biases
            fc1 = tf.nn.dropout(fc1, keep_prob=self.dropout_keep_pro)
            fc1 = tf.nn.relu(fc1)
            
            weigth2 = get_weight(shape=[self.hidden_dim, self.num_classes])
            biases2 = tf.Variable(tf.constant(0.1,dtype=tf.float32, shape = [self.num_classes]))
            self.logits = tf.matmul(fc1, weigth2) + biases2
            
        with tf.name_scope('train_step'):
            print('labels',self.labels.shape)
            print('logits',self.logits.shape)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.labels, logits=self.logits)
            #tf.nn.sparse_softmax_cross_entropy_with_logits(
            loss = tf.reduce_mean(cross_entropy)
            self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(loss)
        
        with tf.name_scope('accuracy'):
            self.y_true = tf.arg_max(self.labels, 1)
            self.y_pre = tf.arg_max(self.logits, 1)
            correct_pre = tf.equal(self.y_pre, self.y_true)
            self.accuracy = tf.reduce_mean(tf.cast(correct_pre,dtype=tf.float32))
        

In [28]:
class Main(object):
    def __init__(self, X, Y, X_val, Y_val):
        self.model = lstm_model()
        self.save_path = 'model/bi_lstm_model/model.ckpt'
        self.X = X
        self.Y = Y
        self.X_val = X_val
        self.Y_val = Y_val
        
    def evaluate(self, sess, x, y):
        y_true = sess.run(self.model.y_true, feed_dict=\
                          {self.model.input_x:x, self.model.input_y:y, self.model.dropout_keep_pro:1.0})
        y_pre = sess.run(self.model.y_pre, feed_dict=\
                         {self.model.input_x:x, self.model.input_y:y, self.model.dropout_keep_pro:1.0})
        score = f1_score(y_true, y_pre, [0.,1.,2.,3.], average=None)
        return 'c0 f1_score:%.3f, c1 f1_score:%.3f, c2 f1_score:%.3f, c3 f1_score:%0.3f' % tuple(score)

    #训练模型
    def train(self, num_epochs):
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            print(X.shape)
            print(Y.shape)
            for epoch in range(num_epochs):
                for xs,ys in tools.next_batch(self.X, self.Y):
                    #print(xs.shape)
                    #print(ys)
                    sess.run(self.model.train_step, feed_dict = \
                             {self.model.input_x:xs, self.model.input_y:ys, self.model.dropout_keep_pro:0.8})
                    #labels = sess.run(model.labels,feed_dict = {model.input_x:xs, model.input_y:ys, model.dropout_keep_pro:0.8})
                    #logits = sess.run(model.logits,feed_dict = {model.input_x:xs, model.input_y:ys, model.dropout_keep_pro:0.8})
                    #acc = sess.run(model.accuracy, feed_dict = {model.input_x:xs, model.input_y:ys, model.dropout_keep_pro:0.8})

                    #print('Iter: ' + str(epoch) + ', Training Accuracy:' + str(acc))
                #self.model.lr *= self.model.lr_decay
                train_acc = sess.run(self.model.accuracy, feed_dict = \
                    {self.model.input_x:self.X, self.model.input_y:self.Y, self.model.dropout_keep_pro:1.0})
                test_acc = sess.run(self.model.accuracy, feed_dict = \
                    {self.model.input_x:self.X_val, self.model.input_y:self.Y_val, self.model.dropout_keep_pro:1.0})
                f1_score_str = self.evaluate(sess, self.X_val, self.Y_val)
                print('Iter: ' + str(epoch) + ', Training Accuracy:' + str(train_acc) + ', Testing Accuracy:'\
                      + str(test_acc) + ' | ' + f1_score_str)
            saver.save(sess=sess, save_path=self.save_path)
            
    def test(self, id2word):
        #saver = tf.train.import_meta_graph(self.save_path + '.meta')  #这个不行
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()
        with tf.Session() as session:
            session.run(init)
            
            saver.restore(session, self.save_path)
            y_pre = session.run(self.model.y_pre, feed_dict=\
                             {self.model.input_x:X_val, self.model.dropout_keep_pro:1.0})
            y_true = session.run(self.model.y_true, feed_dict=\
                          {self.model.input_y:Y_val, self.model.dropout_keep_pro:1.0})
            score = f1_score(y_true, y_pre, [0.,1.,2.,3.], average=None)
            count = 0
            for i in range(len(X_val)):
                line, labels = X_val[i], Y_val[i]
                for j in range(len(line)):
                    #print('y_true', y_true[count])
                    print(id2word[line[j]], np.argmax(labels[j], axis=0), y_pre[count], y_true[count])
                    count += 1
            print('enter')
            print('c0 f1_score:%.3f, c1 f1_score:%.3f, c2 f1_score:%.3f, c3 f1_score:%0.3f' % tuple(score))

In [29]:
main = Main(X, Y, X_val, Y_val)
main.train(5)

(?, 37, 64)
(?, 37, 256)
(?, 256)
labels (?, 4)
logits (?, 4)
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
(4667, 37)
(4667, 37, 4)


/Users/simplzy/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Iter: 0, Training Accuracy:0.741683, Testing Accuracy:0.733136 | c0 f1_score:0.846, c1 f1_score:0.000, c2 f1_score:0.000, c3 f1_score:0.000
Iter: 1, Training Accuracy:0.741683, Testing Accuracy:0.733136 | c0 f1_score:0.846, c1 f1_score:0.000, c2 f1_score:0.000, c3 f1_score:0.000
Iter: 2, Training Accuracy:0.765745, Testing Accuracy:0.767575 | c0 f1_score:0.867, c1 f1_score:0.000, c2 f1_score:0.465, c3 f1_score:0.000
Iter: 3, Training Accuracy:0.781491, Testing Accuracy:0.786741 | c0 f1_score:0.878, c1 f1_score:0.000, c2 f1_score:0.563, c3 f1_score:0.000
Iter: 4, Training Accuracy:0.785434, Testing Accuracy:0.792543 | c0 f1_score:0.882, c1 f1_score:0.000, c2 f1_score:0.593, c3 f1_score:0.000


In [30]:
main = Main(X, Y, X_val, Y_val)
main.test(tools.id2word)

(?, 37, 64)
(?, 37, 256)
(?, 256)
labels (?, 4)
logits (?, 4)
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
INFO:tensorflow:Restoring parameters from model/bi_lstm_model/model.ckpt


/Users/simplzy/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


北 1 2 1
京 2 2 2
朝 2 2 2
阳 2 2 2
浪 2 2 2
漫 2 2 2
约 2 2 2
会 2 2 2
好 2 2 2
去 2 2 2
处 3 0 3
推 0 0 0
荐 0 0 0
两 0 0 0
个 0 0 0
人 0 0 0
最 0 0 0
美 0 0 0
的 0 0 0
时 0 0 0
光 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
2 0 2 0
0 0 2 0
1 0 2 0
4 0 2 0
暑 0 2 0
期 0 2 0
北 1 2 1
京 2 2 2
周 2 2 2
边 2 0 2
亲 2 0 2
水 2 0 2
游 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
佛 0 2 0
山 0 2 0
夏 0 2 0
季 0 2 0
哪 0 2 0
里 0 2 0
好 0 2 0
玩 0 2 0
？ 0 2 0
佛 1 2 1
山 2 0 2
周 2 0 2
边 2 0 2
避 2 0 2
暑 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 

<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
五 0 2 0
一 0 2 0
北 1 2 1
京 2 2 2
周 2 2 2
边 2 2 2
自 2 2 2
驾 2 2 2
游 3 2 3
去 0 2 0
哪 0 0 0
好 0 0 0
? 0 0 0
北 1 0 1
京 2 0 2
五 2 0 2
一 2 0 2
自 2 0 2
驾 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
南 1 2 1
京 2 2 2
秋 2 2 2
季 2 2 2
赏 2 2 2
花 2 2 2
全 2 2 2
攻 2 0 2
略 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
2 0 2 0
0 0 2 0
1 0 2 0
6 0 2 0
杭 1 2 1
州 2 2 2
自 2 2 2
驾 2 2 2
游 3 0 3
攻 0 0 0
略 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD

情 2 0 2
人 2 0 2
节 2 0 2
旅 2 0 2
游 3 0 3
推 0 0 0
荐 0 0 0
地 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
成 1 2 1
都 2 2 2
夏 2 2 2
季 2 2 2
旅 2 2 2
游 2 2 2
好 2 2 2
去 2 0 2
处 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
昆 1 2 1
明 2 2 2
两 2 2 2
日 2 2 2
游 3 2 3
线 0 2 0
路 0 2 0
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0


...

<PAD> 0 0 0
<PAD> 0 0 0
春 0 2 0
节 0 2 0
去 0 2 0
哪 0 2 0
赏 0 2 0
花 0 2 0
好 0 2 0
？ 0 2 0
2 0 2 0
0 0 0 0
1 0 0 0
6 0 0 0
春 1 0 1
节 2 0 2
广 2 0 2
州 2 0 2
周 2 0 2
边 2 0 2
赏 2 0 2
花 2 0 2
游 3 0 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
广 1 2 1
州 2 2 2
十 2 2 2
大 2 2 2
名 2 2 2
山 2 2 2
名 2 2 2
录 3 2 3
广 1 0 1
州 2 0 2
重 2 0 2
阳 2 0 2
登 2 0 2
高 2 0 2
好 2 0 2
地 2 0 2
方 3 0 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
2 0 2 0
0 0 2 0
1 0 2 0
4 0 2 0
年 0 2 0
清 1 2 1
明 2 2 2
节 2 2 2
上 2 2 2
海 2 0 2
周 2 0 2
边 2 0 2
踏 2 0 2
青 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0


<PAD> 0 0 0
金 1 2 1
秋 2 2 2
赏 2 2 2
红 2 2 2
叶 2 2 2
1 2 2 2
0 2 2 2
个 2 2 2
红 2 2 2
叶 2 0 2
观 2 0 2
赏 2 0 2
地 3 0 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
重 1 2 1
庆 2 2 2
文 2 2 2
艺 2 2 2
青 2 2 2
年 2 2 2
周 2 2 2
末 2 2 2
一 2 2 2
日 2 0 2
游 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
杭 1 2 1
州 2 2 2
看 2 2 2
枫 2 2 2
叶 2 2 2
好 2 2 2
地 2 2 2
方 3 2 3
( 0 2 0
介 0 2 0
绍 0 0 0
+ 0 0 0
交 0 0 0
通 0 0 0
攻 0 0 0
略 0 0 0
) 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD

的 2 0 2
1 2 0 2
0 2 0 2
大 2 0 2
目 2 0 2
的 2 0 2
地 3 0 3
单 0 0 0
身 0 0 0
人 0 0 0
士 0 0 0
走 0 0 0
起 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
上 1 2 1
海 2 2 2
周 2 2 2
边 2 2 2
适 2 2 2
合 2 2 2
一 2 2 2
个 2 2 2
人 2 2 2
旅 2 0 2
行 2 0 2
的 2 0 2
1 2 0 2
0 2 0 2
大 2 0 2
目 2 0 2
的 2 0 2
地 3 0 3
单 0 0 0
身 0 0 0
人 0 0 0
士 0 0 0
走 0 0 0
起 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
北 0 2 0
京 0 2 0
哪 0 2 0
里 0 2 0
看 0 2 0
雪 0 2 0
最 0 2 0
好 0 2 0
北 1 2 1
京 2 2 2
看 2 0 2
雪 2 0 2
最 2 0 2
漂 2 0 2
亮 2 0 2
的 2 0 2
地 2 0 2
方 3 0 3
盘 0 0 0
点 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
厦 1 2 1
门 2 2 2
好 2 2 2
玩 2 

<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
昆 1 2 1
明 2 2 2
人 2 2 2
气 2 2 2
景 2 2 2
点 3 2 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
重 1 2 1
庆 2 2 2
2 2 2 2
0 2 2 2
1 2 2 2
8 2 2 2
春 2 2 2
节 2 2 2
自 2 2 2
驾 2 0 2
旅 2 0 2
游 3 0 3
攻 0 0 0
略 0 0 0
( 0 0 0
行 0 0 0
程 0 0 0
+ 0 0 0
游 0 0 0
玩 0 0 0
时 0 0 0
长 0 0 0
) 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
广 1 2 1
州 2 2 2
重 2 2 2
阳 2 2 2
节 2 2 2
登 2 2 2
高 2 2 2
好 2 2 2
去 2 

<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
2 0 2 0
0 0 2 0
1 0 2 0
5 0 2 0
天 1 2 1
津 2 2 2
清 2 2 2
明 2 2 2
节 2 2 2
去 2 2 2
哪 2 0 2
里 2 0 2
玩 2 0 2
旅 2 0 2
游 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
推 0 0 0
荐 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
冬 1 2 1
季 2 2 2
养 2 2 2
生 2 2 2
游 2 2 2
北 2 2 2
京 2 2 2
特 2 2 2
色 2 2 2
温 2 2 2
泉 2 0 2
好 2 0 2
去 2 0 2
处 3 0 3
（ 0 0 0
图 0 0 0
） 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
<PAD> 0 0 0
南 0 2 0
京 0 2 0
4 0 2 0
a 0 2 0
级 0 2 0
景 0 2 0
区 0 2 0
有 0 2 0
哪 0 0 0
些 0 0 0
推 0 0 0
荐 0 0 0
南 1 0 1
京 2 0 2
有 2 0 2
名 2 0 2
的 2 0 2
4 2 0 2
a 2 0 2
级 2 0 2
景 2 0 2
点 3 0 3


In [20]:
### 测试embedding_lookup
'''
tf.reset_default_graph()
x = tf.placeholder(dtype=tf.int32, shape=[None, 37])
embedding = tf.get_variable('embedding', shape=[1745, 64], dtype=tf.float32)
embedding_inputs = tf.nn.embedding_lookup(embedding, x)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for xs,ys in tools.next_batch(X, Y):
        print(xs, xs.shape)
        res_em = sess.run(embedding_inputs, feed_dict = {x:xs})
        print(res_em)
        print(res_em.shape)
        break
'''

"\ntf.reset_default_graph()\nx = tf.placeholder(dtype=tf.int32, shape=[None, 37])\nembedding = tf.get_variable('embedding', shape=[1745, 64], dtype=tf.float32)\nembedding_inputs = tf.nn.embedding_lookup(embedding, x)\ninit = tf.global_variables_initializer()\nwith tf.Session() as sess:\n    sess.run(init)\n    for xs,ys in tools.next_batch(X, Y):\n        print(xs, xs.shape)\n        res_em = sess.run(embedding_inputs, feed_dict = {x:xs})\n        print(res_em)\n        print(res_em.shape)\n        break\n"